In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
% matplotlib inline

In [2]:
test_df = pd.read_csv('../data/test.csv')
test_df.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NaN,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NaN,IR1,Lvl,...,0,185,0,NaN,NaN,NaN,0,7,2009,WD


In [3]:
test_df.shape

(879, 80)

In [4]:
test_df.rename(mapper=lambda x: x.lower().replace(' ', '_'), axis =1, inplace=True)

In [5]:
def series_of_null_features(df):
    return df.loc[:,df.isnull().sum() > 0].isnull().sum()

___
## Cleaning


### What to do with null values
 * Drop mas\_vnr\_*
 * Drop garage\_yr\_built
 * use median for lot\_frontage
 * All other floats use 0 for na
 * Ignore all other cases


In [6]:
series_of_null_features(test_df)

lot_frontage      160
alley             821
mas_vnr_type        1
mas_vnr_area        1
bsmt_qual          25
bsmt_cond          25
bsmt_exposure      25
bsmtfin_type_1     25
bsmtfin_type_2     25
electrical          1
fireplace_qu      422
garage_type        44
garage_yr_blt      45
garage_finish      45
garage_qual        45
garage_cond        45
pool_qc           875
fence             707
misc_feature      838
dtype: int64

In [7]:
# Check that the missing numeric features correspond to the indicator being NA
test_df[test_df.bsmt_exposure.isnull() & ~test_df.bsmt_cond.isnull()].loc[:,['bsmt_exposure', 'bsmt_cond', 'bsmt_qual', 'bsmtfin_type_1', 'bsmtfin_sf_1', 'bsmtfin_type_2','bsmtfin_sf_2', 'bsmt_unf_sf', 'total_bsmt_sf', 'bsmt_full_bath', 'bsmt_half_bath']]

,bsmt_exposure,bsmt_cond,bsmt_qual,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,bsmt_full_bath,bsmt_half_bath


In [8]:
#So the bsmtfin_sf_* == nan's are all from 1 entry, so i'll impute 0 
test_df[test_df.bsmtfin_sf_1.isnull()].loc[:,['bsmt_exposure', 'bsmt_cond', 'bsmt_qual', 'bsmtfin_type_1', 'bsmtfin_sf_1', 'bsmtfin_type_2','bsmtfin_sf_2', 'bsmt_unf_sf', 'total_bsmt_sf', 'bsmt_full_bath', 'bsmt_half_bath', 'bsmt_unf_sf', 'total_bsmt_sf']]

,bsmt_exposure,bsmt_cond,bsmt_qual,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,bsmt_full_bath,bsmt_half_bath,bsmt_unf_sf,total_bsmt_sf


In [9]:
#So the bsmtfin_sf_* == nan's are all from 1 entry, so i'll impute 0 
test_df[test_df.bsmt_full_bath.isnull()].loc[:,['bsmt_exposure', 'bsmt_cond', 'bsmt_qual', 'bsmtfin_type_1', 'bsmtfin_sf_1', 'bsmtfin_type_2','bsmtfin_sf_2', 'bsmt_unf_sf', 'total_bsmt_sf', 'bsmt_full_bath', 'bsmt_half_bath']]

,bsmt_exposure,bsmt_cond,bsmt_qual,bsmtfin_type_1,bsmtfin_sf_1,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,bsmt_full_bath,bsmt_half_bath


In [10]:
test_df.total_bsmt_sf.fillna(0, inplace=True)

In [11]:
test_df.bsmt_unf_sf.fillna(0, inplace=True)

In [12]:
test_df.bsmt_full_bath.fillna(0, inplace=True)

In [13]:
test_df.bsmt_half_bath.fillna(0, inplace=True)

In [14]:
test_df.bsmt_qual.fillna('NA', inplace=True)

In [15]:
test_df.bsmt_cond.fillna('NA', inplace=True)

In [16]:
test_df.bsmt_exposure.fillna('NA', inplace=True)

In [17]:
test_df.bsmtfin_sf_1.fillna(0, inplace=True)

In [18]:
test_df.bsmtfin_type_1.fillna('NA', inplace = True)

In [19]:
test_df.bsmtfin_sf_2.fillna(0, inplace=True)

In [20]:
test_df.bsmtfin_type_2.fillna('NA', inplace=True)

In [21]:
test_df[test_df.garage_finish.isnull() & ~test_df.garage_type.isnull()].loc[:, ['garage_cars','garage_area','garage_type', 'garage_finish', 'garage_qual', 'garage_cond']]

,garage_cars,garage_area,garage_type,garage_finish,garage_qual,garage_cond
765,1.0,360.0,Detchd,NaN,NaN,NaN


I think I'm going to impute NaN for garage_type and then do the transformations on it
___

In [22]:
test_df.garage_area.fillna(0, inplace=True)

In [23]:
test_df.garage_cars.fillna(0, inplace=True)

In [24]:
test_df.garage_cond.fillna('NA', inplace=True)

In [25]:
test_df.garage_finish.fillna('NA', inplace=True)

In [26]:
test_df.garage_qual.fillna('NA', inplace=True)

In [27]:
test_df.garage_type.fillna('NA', inplace=True)

In [28]:
test_df.alley.fillna('NA', inplace=True)

In [29]:
test_df.lot_frontage.fillna(value=test_df.lot_frontage.median(), inplace=True)

In [30]:
test_df.drop(['mas_vnr_type', 'mas_vnr_area', 'garage_yr_blt'], axis = 1, inplace = True)

In [31]:
test_df.fireplace_qu.fillna('NA', inplace=True)

In [32]:
test_df.pool_qc.fillna('NA', inplace=True)

In [33]:
test_df.fence.fillna('NA', inplace=True)

In [34]:
test_df.misc_feature.fillna('NA', inplace=True)

In [35]:
test_df.electrical.fillna('SBrkr', inplace=True)

In [36]:
series_of_null_features(test_df)

Series([], dtype: float64)

# Exporting clean data

In [38]:
test_df.set_index('id', inplace=True)

In [39]:
test_df.to_csv('../data/test_CLEAN.csv')
test_df.head()

,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,...,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type
id,,,,,,,,,,,,,,,,,,,,,
2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,AllPub,...,0,0,0,NA,NA,NA,0,4,2006,WD
2718,905108090,90,RL,68.0,9662,Pave,NA,IR1,Lvl,AllPub,...,0,0,0,NA,NA,NA,0,8,2006,WD
2414,528218130,60,RL,58.0,17104,Pave,NA,IR1,Lvl,AllPub,...,0,0,0,NA,NA,NA,0,9,2006,New
1989,902207150,30,RM,60.0,8520,Pave,NA,Reg,Lvl,AllPub,...,0,0,0,NA,NA,NA,0,7,2007,WD
625,535105100,20,RL,68.0,9500,Pave,NA,IR1,Lvl,AllPub,...,0,185,0,NA,NA,NA,0,7,2009,WD


## So now there should be no null values


### Feature selection
 * Make dummy variables 
   * Remember that MSSubClass will have a value in the train data that wont be in the test data.  Making a dummy variable for this will give a different shape to our train data than to our test data
 * Use LassoCV to determine which coefs should be dropped (see kobe lab) 
 * drop those features manually before making the final model
 * We may want to use `np.log1p()` to remove the skew from the target columns. Don't forget to undo this before turning it in though with `np.expm1()`

In [42]:
test_df.ms_subclass = test_df.ms_subclass.astype('object')

In [44]:
test_df.drop('pid', 1,inplace=True)

In [49]:
cols_need_dummies = test_df.select_dtypes(include='object').columns
test_df_dummied = pd.get_dummies(test_df, columns=cols_need_dummies)

In [51]:
test_df_dummied.shape

(879, 292)

# Now Export the engineered data to use in the model making notebook

In [55]:
test_df_dummied.to_csv('../data/test_ENGINEERED.csv')
test_df_dummied.head()

,lot_frontage,lot_area,overall_qual,overall_cond,year_built,year_remod/add,bsmtfin_sf_1,bsmtfin_sf_2,bsmt_unf_sf,total_bsmt_sf,...,sale_type_COD,sale_type_CWD,sale_type_Con,sale_type_ConLD,sale_type_ConLI,sale_type_ConLw,sale_type_New,sale_type_Oth,sale_type_VWD,sale_type_WD
id,,,,,,,,,,,,,,,,,,,,,
2658,69.0,9142,6,8,1910,1950,0.0,0.0,1020.0,1020.0,...,0,0,0,0,0,0,0,0,0,1
2718,68.0,9662,5,4,1977,1977,0.0,0.0,1967.0,1967.0,...,0,0,0,0,0,0,0,0,0,1
2414,58.0,17104,7,5,2006,2006,554.0,0.0,100.0,654.0,...,0,0,0,0,0,0,1,0,0,0
1989,60.0,8520,5,6,1923,2006,0.0,0.0,968.0,968.0,...,0,0,0,0,0,0,0,0,0,1
625,68.0,9500,6,5,1963,1963,609.0,0.0,785.0,1394.0,...,0,0,0,0,0,0,0,0,0,1


## Now go make a model